In [1]:
from pathlib import Path
import os
import subprocess

In [5]:
# ------------------ CONFIG ------------------
# Try to detect repo root
try:
    REPO_ROOT = Path(__file__).parent.parent.resolve()
except NameError:
    REPO_ROOT = Path(os.getcwd()).parent.resolve()

OUTPUTS_DIR = REPO_ROOT / "outputs"
OUTPUTS_DIR.mkdir(exist_ok=True)

TOPIC_SUMMARY_CSV = OUTPUTS_DIR / "top2vec_topic_summary.csv"
COMMENTS_CSV = OUTPUTS_DIR / "comments_with_top2vec.csv"

Repo root: C:\Users\linna\OneDrive\Documents\Python_Dev\topic-modeling
Outputs folder: C:\Users\linna\OneDrive\Documents\Python_Dev\topic-modeling\outputs
Topic summary exists: True
Comments CSV exists: True


In [7]:
# ------------------ CREATE TEMP STREAMLIT SCRIPT ------------------
APP_SCRIPT = REPO_ROOT / "app/test_app.py"
APP_SCRIPT.parent.mkdir(exist_ok=True)  # ensure app folder exists

app_code = f"""
from pathlib import Path
import streamlit as st
import pandas as pd

DEFAULT_TOPIC_SUMMARY = Path(r"{TOPIC_SUMMARY_CSV}")
DEFAULT_COMMENTS_DF = Path(r"{COMMENTS_CSV}")

st.set_page_config(page_title="Top2Vec Explorer", layout="wide")
st.title("Top2Vec Explorer — Notebook Test")

@st.cache_data
def load_csv(path: Path):
    try:
        return pd.read_csv(path)
    except Exception:
        return None

topic_summary = load_csv(DEFAULT_TOPIC_SUMMARY)
comments_df = load_csv(DEFAULT_COMMENTS_DF)

if topic_summary is None or comments_df is None:
    st.warning("Topic summary or comments CSV not loaded.")
else:
    st.subheader("Topic summary")
    st.dataframe(topic_summary.head(10))
    
    st.subheader("Comments sample")
    st.dataframe(comments_df.head(10))
"""

APP_SCRIPT.write_text(app_code)

923

In [9]:
# ------------------ RUN STREAMLIT ------------------
# This will open the app in your default browser
subprocess.run(["streamlit", "run", str(APP_SCRIPT)])

CompletedProcess(args=['streamlit', 'run', 'C:\\Users\\linna\\OneDrive\\Documents\\Python_Dev\\topic-modeling\\app\\test_app.py'], returncode=4294967295)